In [1]:
import re
import kss
import unicodedata
f = open('4BH00005.txt', encoding="utf8")
i=0
sentence_list = []
while True:
    line = f.readline()
    if not line: break
    if line != '\n':
        i=i+1
        line = re.sub('<.+?>', '', line, 0, re.S)
        line = re.sub('\(.+?\)', '', line, 0, re.S)
        line = re.sub('\[.+?\]', '', line, 0, re.S)
        line = re.sub('\{.+?\}', '', line, 0, re.S)
        line = re.sub('《.+?》', '', line, 0, re.S)
        line = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!『』\‘|\(\)\[\]\<\>`\'…《》\{\}_「」±√]', '', line,0,re.S)
        line = line.lower()
        if len(line) >10:
            sentence_list+= kss.split_sentences(line)
f.close()
print(len(sentence_list))

2264


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(sentence_list)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 13809


In [3]:
sequences = list()

for line in sentence_list:
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[4919, 232],
 [4919, 232, 25],
 [4919, 232, 25, 2797],
 [4919, 232, 25, 2797, 4920],
 [8, 609],
 [8, 609, 610],
 [8, 609, 610, 611],
 [8, 609, 610, 611, 215],
 [8, 609, 610, 611, 215, 1169],
 [8, 609, 610, 611, 215, 1169, 232],
 [8, 609, 610, 611, 215, 1169, 232, 1170]]

In [4]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 1번 단어 : {}'.format(index_to_word[1]))

빈도수 상위 1번 단어 : 있다


In [5]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 90


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [7]:
import numpy as np
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [8]:
print(X[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0 4919]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 4919  232]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  

In [10]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Train on 38358 samples
Epoch 1/200
38358/38358 - 94s - loss: 8.9924 - accuracy: 0.0129
Epoch 2/200
38358/38358 - 92s - loss: 8.4016 - accuracy: 0.0162
Epoch 3/200
38358/38358 - 93s - loss: 8.0643 - accuracy: 0.0215
Epoch 4/200
38358/38358 - 94s - loss: 7.6633 - accuracy: 0.0243
Epoch 5/200
38358/38358 - 95s - loss: 7.1865 - accuracy: 0.0287
Epoch 6/200
38358/38358 - 94s - loss: 6.6929 - accuracy: 0.0361
Epoch 7/200
38358/38358 - 94s - loss: 6.2163 - accuracy: 0.0487
Epoch 8/200
38358/38358 - 92s - loss: 5.7672 - accuracy: 0.0730
Epoch 9/200
38358/38358 - 91s - loss: 5.3524 - accuracy: 0.1159
Epoch 10/200
38358/38358 - 91s - loss: 4.9652 - accuracy: 0.1708
Epoch 11/200


KeyboardInterrupt: 